In [ ]:
pip install paho-mqtt
pip install Flask

In [ ]:
# Importa as bibliotecas necessárias
import paho.mqtt.client as mqtt
import json
from flask import Flask, jsonify
from werkzeug.serving import run_simple
from threading import Thread

# Inicializa a aplicação Flask
app = Flask(__name__)

# Caminho para o arquivo JSON onde os dados serão armazenados
json_file_path = "mqtt_data.json"

# Dicionário para armazenar os dados recebidos do MQTT
data_dict = {}

# Callback para quando o cliente MQTT se conecta ao broker
def on_connect(client, userdata, flags, rc):
    print("Connected with result code " + str(rc))

    # Subscreve nos tópicos relevantes após a conexão ser estabelecida
    topics = ["/TEF/Vita300/attrs/user", "/TEF/Vita300/attrs/validade", "/TEF/Vita300/attrs/estoque"]
    
    for topic in topics:
        client.subscribe(topic)
        print(f"Subscribed to {topic}")

# Callback para quando uma mensagem MQTT é recebida
def on_message(client, userdata, msg):
    global data_dict
    
    # Adiciona os dados ao dicionário usando o nome do tópico como chave
    data_dict[msg.topic] = str(msg.payload)
    
    # Escreve o dicionário atualizado no arquivo JSON
    with open(json_file_path, "w") as json_file:
        json.dump(data_dict, json_file, indent=2)
        print(f"Data written to {json_file_path}")

# Configurações do MQTT (broker e porta)
mqtt_broker_address = "46.17.108.113"
mqtt_port = 1883

# Inicializa o cliente MQTT e configura as callbacks
client = mqtt.Client()
client.on_connect = on_connect
client.on_message = on_message

# Conecta-se ao broker MQTT
client.connect(mqtt_broker_address, mqtt_port, 60)

# Rotas do Flask para obter dados da API
@app.route('/validade', methods=['GET'])
def get_validade():
    return jsonify({'validade': data_dict.get("/TEF/Vita300/attrs/validade", "")})

@app.route('/estoque', methods=['GET'])
def get_estoque():
    return jsonify({'estoque': data_dict.get("/TEF/Vita300/attrs/estoque", "")})

@app.route('/user', methods=['GET'])
def get_user():
    return jsonify({'user': data_dict.get("/TEF/Vita300/attrs/user", "")})

# Função para iniciar o servidor Flask em uma thread separada
def flask_thread():
    run_simple('127.0.0.1', 5000, app, use_reloader=False)

# Ponto de entrada do programa
if __name__ == '__main__':
    # Inicia o servidor Flask em uma thread separada
    flask_thread = Thread(target=flask_thread)
    flask_thread.start()

    # Inicia o loop MQTT para ficar escutando mensagens indefinidamente
    client.loop_forever()
